# Рекомендации на основе содержания

1. Использовать dataset MovieLens
2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:
* TF-IDF на тегах и жанрах
* Средние оценки (+ median, variance, etc.) пользователя и фильма
3. Оценить RMSE на тестовой выборке

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Загружаем датасеты
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies.shape

(9742, 3)

In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [7]:
tags.shape

(3683, 4)

In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
ratings.shape

(100836, 4)

In [10]:
# Поработаем с полем Жанры
movies['genres_new'] = movies['genres'].str.replace('-', ' ').str.replace(' ', '')

In [11]:
movies = movies.drop(['genres'], axis=1)

In [12]:
# Добавим поле Год
movies['year'] = movies['title'].str.slice(start=-5, stop=-1)

In [13]:
# Делаем TF-IDF для жанров

In [14]:
text_genres = movies.genres_new.values
text_genres[:5]

array(['Adventure|Animation|Children|Comedy|Fantasy',
       'Adventure|Children|Fantasy', 'Comedy|Romance',
       'Comedy|Drama|Romance', 'Comedy'], dtype=object)

In [15]:
vectorizer = TfidfVectorizer()

In [16]:
X_genres = vectorizer.fit_transform(text_genres)

In [17]:
vectorizer.vocabulary_

{'adventure': 1,
 'animation': 2,
 'children': 3,
 'comedy': 4,
 'fantasy': 8,
 'romance': 15,
 'drama': 7,
 'action': 0,
 'crime': 5,
 'thriller': 17,
 'horror': 10,
 'mystery': 13,
 'scifi': 16,
 'war': 18,
 'musical': 12,
 'documentary': 6,
 'imax': 11,
 'western': 19,
 'filmnoir': 9,
 'nogenreslisted': 14}

In [18]:
vectorizer.idf_

array([2.67277971, 3.04226778, 3.76757207, 3.68451732, 1.95292831,
       3.09422752, 4.09525948, 1.80361841, 3.52501044, 5.70696754,
       3.29777271, 5.11540016, 4.37017383, 3.83167496, 6.6289563 ,
       2.80842221, 3.2957319 , 2.63733024, 4.23626937, 5.06034038])

In [19]:
print(X_genres.shape)
print(X_genres.toarray())

(9742, 20)
[[0.         0.41684567 0.51622547 ... 0.         0.         0.        ]
 [0.         0.51236121 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.57860574 0.         0.81560738 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [20]:
vectorizer.get_feature_names()

['action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'filmnoir',
 'horror',
 'imax',
 'musical',
 'mystery',
 'nogenreslisted',
 'romance',
 'scifi',
 'thriller',
 'war',
 'western']

In [21]:
# Добавим в таблицу movies наши тэги

In [22]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [23]:
tags = tags[['movieId','tag']].drop_duplicates()

In [24]:
tags['tag_new'] = tags[['movieId', 'tag']].groupby(['movieId'])['tag'].transform(lambda x: ' '.join(x))

In [25]:
tags = tags.drop(columns='tag')

In [26]:
tags = tags.drop_duplicates()

In [27]:
tags.head()

,movieId,tag_new
0,60756,funny Highly quotable will ferrell comedy
3,89774,Boxing story MMA Tom Hardy
6,106782,drugs Leonardo DiCaprio Martin Scorsese Stock ...
9,48516,way too long Leonardo DiCaprio suspense twist ...
10,431,Al Pacino gangster mafia


In [28]:
tags.shape

(1572, 2)

In [29]:
movies['tags_for_tfidf'] = movies.apply(lambda r: ' '.join(tags[tags['movieId']==r['movieId']]['tag_new'].values), axis=1)

In [30]:
movies.head()

,movieId,title,genres_new,year,tags_for_tfidf
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,pixar fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,fantasy magic board game Robin Williams game
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,moldy old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,
4,5,Father of the Bride Part II (1995),Comedy,1995,pregnancy remake


In [31]:
# Делаем TF-IDF для тэгов

In [32]:
tag_strings = movies['tags_for_tfidf'].values
tag_strings[:5]

array(['pixar fun', 'fantasy magic board game Robin Williams game',
       'moldy old', '', 'pregnancy remake'], dtype=object)

In [33]:
vectorizer_tags = TfidfVectorizer()

In [34]:
X_tags = vectorizer_tags.fit_transform(tag_strings)

In [35]:
#vectorizer_tags.vocabulary_

In [36]:
print(X_tags.shape)
print(X_tags.toarray())

(9742, 1744)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [37]:
# Пришлось убрать на этом этапе часть фичей, так как все время был вылет по памяти
X_tags = X_tags.toarray()[:, :400]
X_tags.shape

(9742, 400)

In [38]:
# Объединяем в один датасет жанры и тэги после TF-IDF

In [39]:
X_genres_tfidf = pd.DataFrame(X_genres.toarray(), columns=vectorizer.get_feature_names())
X_genres_tfidf.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [40]:
X_tags_tfidf = pd.DataFrame(X_tags, columns=vectorizer_tags.get_feature_names()[:400])
X_tags_tfidf.shape

(9742, 400)

In [ ]:
#X_tags_tfidf = pd.DataFrame(X_tags.toarray(), columns=vectorizer_tags.get_feature_names())
#X_tags_tfidf.shape

In [41]:
movies1 = movies.join(X_genres_tfidf, how='left')
movies1.head()

,movieId,title,genres_new,year,tags_for_tfidf,action,adventure,animation,children,comedy,...,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,pixar fun,0.0,0.416846,0.516225,0.504845,0.267586,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,fantasy magic board game Robin Williams game,0.0,0.512361,0.000000,0.620525,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,moldy old,0.0,0.000000,0.000000,0.000000,0.570915,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,,0.0,0.000000,0.000000,0.000000,0.505015,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,1995,pregnancy remake,0.0,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [42]:
movies_tfidf = movies1.join(X_tags_tfidf, how='left', lsuffix='t')
movies_tfidf.head()

,movieId,title,genres_new,year,tags_for_tfidf,actiont,adventuret,animationt,childrent,comedyt,...,cusack,cuts,cyberpunk,cyborg,cynical,dahl,damon,dan,dance,dancing
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,pixar fun,0.0,0.416846,0.516225,0.504845,0.267586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,fantasy magic board game Robin Williams game,0.0,0.512361,0.000000,0.620525,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,moldy old,0.0,0.000000,0.000000,0.000000,0.570915,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,,0.0,0.000000,0.000000,0.000000,0.505015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,1995,pregnancy remake,0.0,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# movies_tfidf - наш объединенный датасет

In [44]:
movies_tfidf.shape

(9742, 425)

In [45]:
# Выбираем пользователя для которого будем делать предсказание оценки
ratings.groupby('userId').count().sort_values('rating', ascending=False).head(5)

,movieId,rating,timestamp
userId,,,
414,2698,2698,2698
599,2478,2478,2478
474,2108,2108,2108
448,1864,1864,1864
274,1346,1346,1346


In [46]:
User = 448

In [47]:
# Добавляем новые фичи
ratings_with_mean = ratings.groupby('movieId')['rating'].agg(['mean', 'var', 'count']).reset_index()
ratings_with_mean=ratings_with_mean.fillna(0)
ratings_with_mean.head()

,movieId,mean,var,count
0,1,3.920930,0.696990,215
1,2,3.431818,0.777419,110
2,3,3.259615,1.112651,52
3,4,2.357143,0.726190,7
4,5,3.071429,0.822917,49


In [48]:
ratings_with_mean = ratings.merge(ratings_with_mean)
ratings_with_mean.head()

,userId,movieId,rating,timestamp,mean,var,count
0,1,1,4.0,964982703,3.92093,0.69699,215
1,5,1,4.0,847434962,3.92093,0.69699,215
2,7,1,4.5,1106635946,3.92093,0.69699,215
3,15,1,2.5,1510577970,3.92093,0.69699,215
4,17,1,4.5,1305696483,3.92093,0.69699,215


In [49]:
df_finish = ratings_with_mean.merge(movies_tfidf)
df_finish.head()

,userId,movieId,rating,timestamp,mean,var,count,title,genres_new,year,...,cusack,cuts,cyberpunk,cyborg,cynical,dahl,damon,dan,dance,dancing
0,1,1,4.0,964982703,3.92093,0.69699,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,1,4.0,847434962,3.92093,0.69699,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,1,4.5,1106635946,3.92093,0.69699,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,1,2.5,1510577970,3.92093,0.69699,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17,1,4.5,1305696483,3.92093,0.69699,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
df_for_user = df_finish[df_finish['userId']==User]
df_for_user.head()

,userId,movieId,rating,timestamp,mean,var,count,title,genres_new,year,...,cusack,cuts,cyberpunk,cyborg,cynical,dahl,damon,dan,dance,dancing
154,448,1,5.0,1019126661,3.920930,0.696990,215,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,448,3,3.0,1019128536,3.259615,1.112651,52,Grumpier Old Men (1995),Comedy|Romance,1995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
526,448,47,4.0,1019132386,3.975369,0.850875,203,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
729,448,50,4.0,1064741727,4.237745,0.641475,204,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
843,448,101,3.5,1076328586,3.782609,1.086957,23,Bottle Rocket (1996),Adventure|Comedy|Crime|Romance,1996,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
df_for_user = df_for_user.drop(columns=['title', 'genres_new', 'timestamp', 'tags_for_tfidf'])

In [52]:
df_for_user.loc[:, df_for_user.dtypes == object].head()

,year
154,1995
252,1995
526,1995
729,1995
843,1996


In [53]:
y = df_for_user['rating']
y.shape

(1864,)

In [54]:
X = df_for_user.drop(columns=['rating','userId', 'movieId'])
X.shape

(1864, 424)

In [55]:
# Для поля Год делаем dummies (выше нашли, что у нас одна категориальная переменная)
X_dummies = pd.get_dummies(X, columns=['year'])

In [56]:
X_dummies.head()

,mean,var,count,actiont,adventuret,animationt,childrent,comedyt,crimet,documentary,...,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017
154,3.920930,0.696990,215,0.0,0.416846,0.516225,0.504845,0.267586,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
252,3.259615,1.112651,52,0.0,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
526,3.975369,0.850875,203,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
729,4.237745,0.641475,204,0.0,0.000000,0.000000,0.000000,0.000000,0.553854,0.0,...,0,0,0,0,0,0,0,0,0,0
843,3.782609,1.086957,23,0.0,0.550590,0.000000,0.000000,0.353441,0.559994,0.0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
# Разбиваем данные на тест и трейн

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.2)

In [60]:
# Делаем масштабирование

In [61]:
from sklearn.preprocessing import StandardScaler

In [62]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [63]:
# Попробуем модель RandomForestRegressor с max_depth=8 и n_estimators=70
# гиперпараметры определены по GridSearch, но повторно запускать его не решилась)

In [64]:
from sklearn.ensemble import RandomForestRegressor

In [65]:
model = RandomForestRegressor(max_depth=8, n_estimators=70)

In [66]:
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=8, n_estimators=70)

In [67]:
y_pred = model.predict(X_test)

In [68]:
from sklearn.metrics import r2_score, mean_squared_error

In [69]:
r2_score(y_test, y_pred)

0.6023683258240617

In [70]:
mean_squared_error(y_test, y_pred)**0.5

0.6414460200695685

In [ ]:
#from sklearn.model_selection import GridSearchCV

In [ ]:
#grid_params = {'max_depth': list(range(1, 20)), 'n_estimators': [30, 50, 70, 100, 120]}

In [ ]:
#forest = GridSearchCV(RandomForestRegressor(), param_grid=grid_params, cv=10, scoring = 'r2')
#forest.fit(X_train, y_train)

In [ ]:
#y_pred = forest.best_estimator_.predict(X_test)

In [ ]:
#forest.best_params_

In [ ]:
#r2_score(y_test, y_pred)

In [ ]:
#mean_squared_error(y_test, y_pred)**0.5

In [ ]:
# Попробуем модель LGBMRegressor

In [71]:
from lightgbm import LGBMRegressor

In [72]:
lgbm = LGBMRegressor()

In [73]:
lgbm.fit(X_train, y_train)

LGBMRegressor()

In [74]:
y_pred = lgbm.predict(X_test)

In [75]:
r2_score(y_test, y_pred)

0.5798403560072591

In [76]:
mean_squared_error(y_test, y_pred)**0.5

0.6593663766268242

In [77]:
# Попробуем модель SVR

In [78]:
from sklearn.svm import SVR

In [79]:
svr = SVR()

In [80]:
svr.fit(X_train, y_train)

SVR()

In [81]:
y_pred = svr.predict(X_test)

In [82]:
r2_score(y_test, y_pred)

0.48939033974182855

In [83]:
mean_squared_error(y_test, y_pred)**0.5

0.7268823816934834

In [ ]:
# Получилось, что лучше всего отработал RandomForestRegressor RMSE=0.64